# Counting for MicroNet Challenge Score

Altough we thoroughly specified the calculation results of our model in `readme.md` file, we double checked the score using refactored counting code which has uploaded by the organizers: [official counting code](https://github.com/google-research/google-research/blob/master/micronet_challenge/counting.py)

We defined our `read model` module and `counting module` for newly defined operations. Since our backbone architecture is following EfficientNet, which was the example of official counting code, we examined our counting code for EfficientNet.

The result from our counting code was same with official counting code for EfficientNet. Below are the results for the score for our model and EfficientNet

### 1. Our model

In [1]:
import os
import sys
import math
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.models as models

from data_utils import *
from train_tools import *
from models import *
from counting import *
from pthflops import count_ops

opt = ConfLoader('./Config/test.json').opt

dataloaders, dataset_sizes = cifar_100_setter(batch_size=opt.data.batch_size, 
                                              valid_size=opt.data.valid_size,
                                              root=opt.data.root,
                                              fixed_valid=opt.data.fixed_valid,
                                              autoaugment=opt.data.autoaugment,
                                              aug_policy=opt.data.aug_policy)

avail_resource = 2.0
resolution_coefficient = 1.4
resolution_coefficient = round(math.pow(resolution_coefficient, avail_resource), 2)

blocks_args1 = [
    'r1_k3_s1_e1_i24_o16_se0.20', 
    'r1_k3_s1_e6_i16_o24_se0.20', 
    'r2_k3_s2_e6_i24_o40_se0.20',  
    'r1_k3_s1_e6_i40_o56_se0.20',  
    'r2_k3_s1_e6_i56_o72_se0.20',  
    'r2_k3_s2_e6_i72_o88_se0.20',  
    'r1_k3_s1_e6_i88_o104_se0.20'
]

blocks_args1, global_params1 = efficientnet(blocks_args=blocks_args1,
                                           resolution_coefficient=resolution_coefficient,
                                           width_coefficient=0.9, 
                                           depth_coefficient=1, 
                                           image_size=opt.model.param.image_size, 
                                           num_classes=opt.model.param.num_classes)

model = EfficientNet(blocks_args1, 
                     global_params1)

model.to(opt.trainhandler.device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


EfficientNet(
  (upsample): Upsample(size=63, mode=nearest)
  (_conv_stem): MaskedConv2dStaticSamePadding(
    3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_conv): MaskedConv2dStaticSamePadding(
        24, 24, kernel_size=(3, 3), stride=[1, 1], groups=24, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (_se_reduce): MaskedConv2dStaticSamePadding(
        24, 4, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): MaskedConv2dStaticSamePadding(
        4, 24, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): MaskedConv2dStaticSamePadding(


### 2. Score of our Model

In [3]:
import collections
from counting import *

# add_bits_base=32, since 32 bit adds count 1 add.
# mul_bits_base=32, since multiplications with 32 bit input count 1 multiplication.
conv_stem = {'kernel': 3, 'stride': 2, 'out_channel': 24}
last_ops = {'out_channel': 150, 'num_classes': 100}
activation = 'swish'
input_size = int(math.ceil(32 * resolution_coefficient))
use_bias = False

counter = MicroNetCounter(conv_stem, blocks_args1, global_params1, last_ops, activation, input_size, use_bias, add_bits_base=32, mul_bits_base=32)

# Constants
INPUT_BITS = 16
ACCUMULATOR_BITS = 32
PARAMETER_BITS = INPUT_BITS
SUMMARIZE_BLOCKS = True
SPARSITY = 0.6

params, flops, _, _ = counter.print_summary(SPARSITY, PARAMETER_BITS, ACCUMULATOR_BITS, INPUT_BITS, summarize_blocks=SUMMARIZE_BLOCKS)

print('#'*50)
print("flops: {:.6f}M, params: {:.6f}MBytes".format(flops, params))
print('score: {:.6f} + {:.6f} = {:.6f}'.format(flops/(10490), params/(36.5*4), flops/(10490) + params/(36.5*4)))

op_name                     inp_size   kernel_size   in channels  out channels  params(MBytes)   mults(M)    adds(M)     MFLOPS
_conv_stem                        63             3             3            24           0.001      0.159      0.249      0.408
block_0                           31            -1            24            -1           0.001      0.162      0.238      0.400
block_1                           31            -1            16            -1           0.005      1.227      1.878      3.105
block_2                           31            -1            24            -1           0.011      1.255      1.987      3.242
block_3                           15            -1            40            -1           0.024      1.151      2.023      3.174
block_4                           15            -1            40            -1           0.026      1.238      2.140      3.378
block_5                           15            -1            48            -1           0.038      1.79

### 3. EfficientNet-b0

In [3]:
import os
import sys
import math
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.models as models

from data_utils import *
from train_tools import *
from models import *
from counting import *
from pthflops import count_ops

opt = ConfLoader('./Config/test.json').opt

dataloaders, dataset_sizes = cifar_100_setter(batch_size=opt.data.batch_size, 
                                              valid_size=opt.data.valid_size,
                                              root=opt.data.root,
                                              fixed_valid=opt.data.fixed_valid,
                                              autoaugment=opt.data.autoaugment,
                                              aug_policy=opt.data.aug_policy)

blocks_args = [
    'r1_k3_s1_e1_i32_o16_se0.25', 
    'r2_k3_s2_e6_i16_o24_se0.25', 
    'r2_k5_s2_e6_i24_o40_se0.25',  
    'r3_k3_s2_e6_i40_o80_se0.25',  
    'r3_k5_s1_e6_i80_o112_se0.25',  
    'r4_k5_s2_e6_i112_o192_se0.25',  
    'r1_k3_s1_e6_i192_o320_se0.25'
]

blocks_args, global_params = efficientnet(blocks_args=blocks_args,
                                          resolution_coefficient=1,
                                          width_coefficient=1, 
                                          depth_coefficient=1, 
                                          image_size=224, 
                                          num_classes=1000)

model = EfficientNet(blocks_args, 
                     global_params)

model.to(opt.trainhandler.device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


EfficientNet(
  (_conv_stem): MaskedConv2dStaticSamePadding(
    3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): SymmetricPad2d(
      (padding1): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
      (padding2): ZeroPad2d(padding=(1, 0, 0, 1), value=0.0)
      (padding3): ZeroPad2d(padding=(0, 1, 1, 0), value=0.0)
      (padding4): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
  )
  (_bn0): BatchNorm(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_conv): MaskedConv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (_se_reduce): MaskedConv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Ma

### 4. Score of EfficientNet-b0

In [4]:
from counting import *

# add_bits_base=32, since 32 bit adds count 1 add.
# mul_bits_base=32, since multiplications with 32 bit input count 1 multiplication.
conv_stem = {'kernel': 3, 'stride': 2, 'out_channel': 32}
last_ops = {'out_channel': 1280, 'num_classes': 1000}
activation = 'swish'
input_size = 224
use_bias = True

counter = MicroNetCounter(conv_stem, blocks_args, global_params, last_ops, activation, input_size, use_bias, add_bits_base=32, mul_bits_base=32)

# Constants
INPUT_BITS = 16
ACCUMULATOR_BITS = 32
PARAMETER_BITS = INPUT_BITS
SUMMARIZE_BLOCKS = True
SPARSITY = 0.0

params, flops, _, _ = counter.print_summary(SPARSITY, PARAMETER_BITS, ACCUMULATOR_BITS, INPUT_BITS, summarize_blocks=SUMMARIZE_BLOCKS)

print('#'*50)
print("flops: {:.4f}M, params: {:.4f}MBytes".format(flops, params))
print('score: {:.4f} + {:.4f} = {:.4f}'.format(flops/(10490), params/(36.5*4), flops/(10490) + params/(36.5*4)))


op_name                     inp_size   kernel_size   in channels  out channels  params(MBytes)   mults(M)    adds(M)     MFLOPS
_conv_stem                       224             3             3            32           0.002      6.021     11.239     17.261
block_0                          112            -1            32            -1           0.003      5.821     10.839     16.659
block_1                          112            -1            16            -1           0.012     17.010     31.010     48.020
block_2                           56            -1            24            -1           0.021     14.452     27.548     42.000
block_3                           56            -1            24            -1           0.030      9.992     18.855     28.848
block_4                           28            -1            40            -1           0.062     10.540     20.514     31.054
block_5                           28            -1            40            -1           0.073      6.23

### 5. Further Validation

Following parts are definitions for named tuples in official counting code.

We examined our counting code for EfficientNet to verify the result of `read model` in official code and our code is same.

In [5]:
Conv2D = collections.namedtuple(
    'Conv2D', ['input_size', 'kernel_shape', 'strides', 'padding', 'use_bias',
               'activation'])

"""Operation definition for 2D depthwise convolution.
Only difference compared to Conv2D is the kernel_shape[3] = 1.
"""
DepthWiseConv2D = collections.namedtuple(
    'DepthWiseConv2D', ['input_size', 'kernel_shape', 'strides', 'padding',
                        'use_bias', 'activation'])

"""Operation definition for Global Average Pooling.
Attributes:
  input_size: int, Dimensions of the input image (square assumed).
  n_channels: int, Number of output dimensions.
"""
GlobalAvg = collections.namedtuple('GlobalAvg', ['input_size', 'n_channels'])

"""Operation definitions for elementwise multiplication and addition.
Attributes:
  input_size: int, Dimensions of the input image (square assumed).
  n_channels: int, Number of output dimensions.
"""
Scale = collections.namedtuple('Scale', ['input_size', 'n_channels'])
Add = collections.namedtuple('Add', ['input_size', 'n_channels'])


"""Operation definitions for elementwise multiplication and addition.
Attributes:
  kernel_shape: list, of length 2. Shape of the weight matrix.
  use_bias: bool, if true a bias term is added to the output.
  activation: str, type of activation applied to the output.
"""
FullyConnected = collections.namedtuple(
    'FullyConnected', ['kernel_shape', 'use_bias', 'activation'])

#### Result of read_model for EfficientNet-b0 (from official counting code)

In [6]:
official = [('_conv_stem', Conv2D(input_size=224, kernel_shape=[3, 3, 3, 32], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('block_0', [('_depthwise_conv', DepthWiseConv2D(input_size=112, kernel_shape=[3, 3, 32, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=112, n_channels=32)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 32, 8], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 8, 32], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=112, n_channels=32)), ('_project_conv', Conv2D(input_size=112, kernel_shape=[1, 1, 32, 16], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_1', [('_expand_conv', Conv2D(input_size=112, kernel_shape=[1, 1, 16, 96], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=112, kernel_shape=[3, 3, 96, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=56, n_channels=96)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 96, 4], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 4, 96], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=56, n_channels=96)), ('_project_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 96, 24], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_2', [('_expand_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 24, 144], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=56, kernel_shape=[3, 3, 144, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=56, n_channels=144)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 144, 6], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 6, 144], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=56, n_channels=144)), ('_project_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 144, 24], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=56, n_channels=144))]), ('block_3', [('_expand_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 24, 144], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=56, kernel_shape=[5, 5, 144, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=28, n_channels=144)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 144, 6], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 6, 144], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=28, n_channels=144)), ('_project_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 144, 40], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_4', [('_expand_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 40, 240], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=28, kernel_shape=[5, 5, 240, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=28, n_channels=240)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 240, 10], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 10, 240], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=28, n_channels=240)), ('_project_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 240, 40], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=28, n_channels=240))]), ('block_5', [('_expand_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 40, 240], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=28, kernel_shape=[3, 3, 240, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=240)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 240, 10], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 10, 240], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=240)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 240, 80], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_6', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 80, 480], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[3, 3, 480, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=480)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 480, 20], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 20, 480], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=480)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 480, 80], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=480))]), ('block_7', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 80, 480], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[3, 3, 480, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=480)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 480, 20], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 20, 480], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=480)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 480, 80], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=480))]), ('block_8', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 80, 480], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 480, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=480)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 480, 20], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 20, 480], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=480)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 480, 112], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_9', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 112, 672], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 672, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=672)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 672, 28], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 28, 672], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=672)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 672, 112], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=672))]), ('block_10', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 112, 672], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 672, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=672)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 672, 28], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 28, 672], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=672)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 672, 112], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=672))]), ('block_11', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 112, 672], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 672, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=672)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 672, 28], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 28, 672], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=672)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 672, 192], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_12', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[5, 5, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 192], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=7, n_channels=1152))]), ('block_13', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[5, 5, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 192], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=7, n_channels=1152))]), ('block_14', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[5, 5, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 192], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=7, n_channels=1152))]), ('block_15', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[3, 3, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 320], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('_conv_head', Conv2D(input_size=7, kernel_shape=[1, 1, 320, 1280], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_avg_pooling', GlobalAvg(input_size=7, n_channels=1280)), ('_fc', FullyConnected(kernel_shape=[1280, 1000], use_bias=True, activation=None))]

#### Result of read_model for EfficientNet-b0 (from our counting code)

In [7]:
mine = [('_conv_stem', Conv2D(input_size=224, kernel_shape=[3, 3, 3, 32], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('block_0', [('_depthwise_conv', DepthWiseConv2D(input_size=112, kernel_shape=[3, 3, 32, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=112, n_channels=32)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 32, 8], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 8, 32], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=112, n_channels=32)), ('_project_conv', Conv2D(input_size=112, kernel_shape=[1, 1, 32, 16], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_1', [('_expand_conv', Conv2D(input_size=112, kernel_shape=[1, 1, 16, 96], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=112, kernel_shape=[3, 3, 96, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=56, n_channels=96)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 96, 4], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 4, 96], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=56, n_channels=96)), ('_project_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 96, 24], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_2', [('_expand_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 24, 144], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=56, kernel_shape=[3, 3, 144, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=56, n_channels=144)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 144, 6], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 6, 144], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=56, n_channels=144)), ('_project_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 144, 24], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=56, n_channels=144))]), ('block_3', [('_expand_conv', Conv2D(input_size=56, kernel_shape=[1, 1, 24, 144], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=56, kernel_shape=[5, 5, 144, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=28, n_channels=144)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 144, 6], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 6, 144], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=28, n_channels=144)), ('_project_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 144, 40], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_4', [('_expand_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 40, 240], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=28, kernel_shape=[5, 5, 240, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=28, n_channels=240)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 240, 10], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 10, 240], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=28, n_channels=240)), ('_project_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 240, 40], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=28, n_channels=240))]), ('block_5', [('_expand_conv', Conv2D(input_size=28, kernel_shape=[1, 1, 40, 240], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=28, kernel_shape=[3, 3, 240, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=240)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 240, 10], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 10, 240], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=240)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 240, 80], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_6', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 80, 480], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[3, 3, 480, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=480)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 480, 20], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 20, 480], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=480)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 480, 80], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=480))]), ('block_7', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 80, 480], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[3, 3, 480, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=480)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 480, 20], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 20, 480], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=480)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 480, 80], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=480))]), ('block_8', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 80, 480], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 480, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=480)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 480, 20], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 20, 480], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=480)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 480, 112], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_9', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 112, 672], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 672, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=672)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 672, 28], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 28, 672], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=672)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 672, 112], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=672))]), ('block_10', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 112, 672], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 672, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=14, n_channels=672)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 672, 28], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 28, 672], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=14, n_channels=672)), ('_project_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 672, 112], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=14, n_channels=672))]), ('block_11', [('_expand_conv', Conv2D(input_size=14, kernel_shape=[1, 1, 112, 672], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=14, kernel_shape=[5, 5, 672, 1], strides=(2, 2), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=672)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 672, 28], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 28, 672], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=672)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 672, 192], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('block_12', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[5, 5, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 192], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=7, n_channels=1152))]), ('block_13', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[5, 5, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 192], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=7, n_channels=1152))]), ('block_14', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[5, 5, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 192], strides=(1, 1), padding='same', use_bias=True, activation=None)), ('_skip_add', Add(input_size=7, n_channels=1152))]), ('block_15', [('_expand_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 192, 1152], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_depthwise_conv', DepthWiseConv2D(input_size=7, kernel_shape=[3, 3, 1152, 1], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_reduce_mean', GlobalAvg(input_size=7, n_channels=1152)), ('_se_reduce', Conv2D(input_size=1, kernel_shape=[1, 1, 1152, 48], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_se_expand', Conv2D(input_size=1, kernel_shape=[1, 1, 48, 1152], strides=(1, 1), padding='same', use_bias=True, activation='sigmoid')), ('_se_scale', Scale(input_size=7, n_channels=1152)), ('_project_conv', Conv2D(input_size=7, kernel_shape=[1, 1, 1152, 320], strides=(1, 1), padding='same', use_bias=True, activation=None))]), ('_conv_head', Conv2D(input_size=7, kernel_shape=[1, 1, 320, 1280], strides=(1, 1), padding='same', use_bias=True, activation='swish')), ('_avg_pooling', GlobalAvg(input_size=7, n_channels=1280)), ('_fc', FullyConnected(kernel_shape=[1280, 1000], use_bias=True, activation=None))]

#### Verification

In [8]:
official == mine

True